# Import All Libaries

In [10]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()

import pyterrier as pt
from tqdm import tqdm
from jnius import autoclass
import gzip
import json


# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
tira = Client()

# Load the Dataset and the Index

In [11]:
# The dataset: antique-test-20230107-training because only this dataset works with docT5query
dataset = 'antique-test-20230107-training'
pt_dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset}')
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

## Document Expansion by Query Prediction with docT5query
The basic idea is to train a model, that when given an input document, generates questions that the document might answer (or more broadly, queries for which the document might be relevant). These predicted questions (or queries) are then appended to the original documents, which are then indexed as before. The docT5query model gets its name from the use of T5 as the expansion model.

The primary advantage of this approach is that expensive neural inference is pushed to indexing time, which means that "bag of words" queries against an inverted index built on the augmented document collection are only slightly slower (due to longer documents) — but the retrieval results are much better.

First we check, if our corpus has a high recall or a lower. Our Corpus in this case is the union of the IR Anthology and the ACL Anthology. The recall may change if we use another corpus.

In [12]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pt.Experiment(
    retr_systems=[bm25],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25'],
    eval_metrics=['recall_1000']
)

,name,recall_1000
0,BM25,0.788732


As we can see, we have already a high recall. This is important for the way we implement the docT5query.
More information about the implementation in the [Tutorial](https://github.com/tira-io/teaching-ir-with-shared-tasks/blob/main/tutorials/tutorial-doc-t5-query.ipynb).

In [13]:
def doc_t5_query(dataset):
    docs = tira.get_run_output('ir-benchmarks/seanmacavaney/DocT5Query', dataset) + '/documents.jsonl.gz'
    with gzip.open(docs, 'rt') as f:
        for l in tqdm(f):
            l = json.loads(l)
            l['text'] = l['querygen']
            l['docno'] = l['doc_id']
            del l['doc_id']
            del l['querygen']
            yield l

def doc_t5_query_index(dataset):
    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(doc_t5_query(dataset))
    return pt.IndexFactory.of(index_ref)

In [14]:
indexD = doc_t5_query_index(dataset)

0it [00:00, ?it/s]

3623it [00:00, 6462.00it/s]

14:38:01.426 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (3052500_1) - further warnings are suppressed


403666it [00:32, 12546.55it/s]


14:38:35.863 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 113 empty documents


In [15]:
docs_retrieved_by_bm25 = {}

bm25_result = bm25(pt_dataset.get_topics('title'))

for _, i in tqdm(bm25_result.iterrows()):
    qid, docno = str(i['qid']), str(i['docno'])

    if qid not in docs_retrieved_by_bm25:
        docs_retrieved_by_bm25[qid] = set()
    
    docs_retrieved_by_bm25[qid].add(docno)

188633it [00:07, 23659.85it/s]


In [16]:
omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]
omit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)

In [17]:
bm25_doct5query = pt.BatchRetrieve(indexD, wmodel="BM25")
bm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs

## Evaluation

In [18]:
pt.Experiment(
    retr_systems=[bm25, bm25_doct5query, bm25_doct5query_new],
    topics=pt_dataset.get_topics(),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'DocT5Query >> BM25', '(DocT5Query >> BM25) >> Retrieved_Docs'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut_10', 'recip_rank']
)
 


There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,recall_1000,ndcg_cut_5,ndcg_cut_10,recip_rank
0,BM25,0.788732,0.529428,0.510402,0.934803
1,DocT5Query >> BM25,0.534685,0.399011,0.348678,0.793546
2,(DocT5Query >> BM25) >> Retrieved_Docs,0.019399,0.015763,0.012670,0.056205


DocT5Query bringt uns im normalen Dataset nicht so viel, also lassen wir es weg